In [2]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table

In [3]:
AAD_TENANT_ID = "6babcaad-604b-40ac-a9d7-9fd97c0b779f"
KUSTO_URI = "https://llm-runners.eastus.kusto.windows.net"
KUSTO_INGEST_URI = "https://ingest-llm-runners.eastus.kusto.windows.net"
KUSTO_DATABASE = "llm-runner"

In [4]:
KCSB_DATA  = KustoConnectionStringBuilder.with_aad_user_token_authentication(KUSTO_URI,"..------")
KCSB_INGEST   = KustoConnectionStringBuilder.with_aad_user_token_authentication(KUSTO_INGEST_URI, "..-------")

In [6]:
KUSTO_CLIENT = KustoClient(KCSB_DATA)
CREATE_TABLE_COMMAND = ".create table TestEvents (PreciseTimeStamp: datetime, n_prompt: int, n_gen: int)"

RESPONSE = KUSTO_CLIENT.execute_mgmt(KUSTO_DATABASE, CREATE_TABLE_COMMAND)

dataframe_from_result_table(RESPONSE.primary_results[0])

,TableName,Schema,DatabaseName,Folder,DocString
0,TestEvents,"{""Name"":""TestEvents"",""OrderedColumns"":[{""Name""...",llm-runner,None,None


In [7]:
CREATE_MAPPING_COMMAND = """.create table TestEvents ingestion csv mapping 'TestEvents_CSV_Mapping' '[{"Name":"PreciseTimeStamp","datatype":"datetime","Ordinal":0}, {"Name":"n_prompt","datatype":"int","Ordinal":1},{"Name":"n_gen","datatype":"int","Ordinal":2}]'"""

RESPONSE = KUSTO_CLIENT.execute_mgmt(KUSTO_DATABASE, CREATE_MAPPING_COMMAND)

dataframe_from_result_table(RESPONSE.primary_results[0])

,Name,Kind,Mapping,LastUpdatedOn,Database,Table
0,TestEvents_CSV_Mapping,Csv,"[{""Name"":""PreciseTimeStamp"",""DataType"":""dateti...",2024-04-16 03:35:15.163397600+00:00,llm-runner,TestEvents


In [35]:
import io
from datetime import datetime, timezone
p_time = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S.%f")
n_prompt= 123
n_gen=456
content = io.StringIO(f"{p_time},{n_prompt},{n_gen}")
    


In [20]:
from azure.kusto.data import DataFormat
from azure.kusto.ingest import QueuedIngestClient, IngestionProperties, FileDescriptor, BlobDescriptor, ReportLevel, ReportMethod

INGESTION_CLIENT = QueuedIngestClient(KCSB_INGEST)

INGESTION_PROPERTIES = IngestionProperties(database=KUSTO_DATABASE, table="TestEvents", data_format=DataFormat.CSV,
                                           ingestion_mapping_reference="TestEvents_CSV_Mapping", additional_properties={'ignoreFirstRecord': 'true'})

INGESTION_CLIENT.ingest_from_stream(content, ingestion_properties=INGESTION_PROPERTIES)

IngestionResult(status=IngestionStatus.QUEUED, database=llm-runner, table=TestEvents, source_id=ff17a293-a1d9-40f8-9c25-a4d39fa26f79, blob_uri=https://hdjkstrldllmrunners01.blob.core.windows.net/20240416-ingestdata-e5c334ee145d4b4-0/llm-runner__TestEvents__ff17a293-a1d9-40f8-9c25-a4d39fa26f79__stream.gz)

In [36]:
INGESTION_PROPERTIES = IngestionProperties(database=KUSTO_DATABASE, table="TestEvents", data_format=DataFormat.CSV,
                                           ingestion_mapping_reference="TestEvents_CSV_Mapping", additional_properties={'ignoreFirstRecord': 'false'})

INGESTION_CLIENT.ingest_from_stream(content, ingestion_properties=INGESTION_PROPERTIES)

IngestionResult(status=IngestionStatus.QUEUED, database=llm-runner, table=TestEvents, source_id=02035713-6f38-4b36-a725-5f68d8052ba6, blob_uri=https://pwrkstrldllmrunners00.blob.core.windows.net/20240416-ingestdata-e5c334ee145d4b4-0/llm-runner__TestEvents__02035713-6f38-4b36-a725-5f68d8052ba6__stream.gz)

In [22]:
with open("/tmp/hi.csv", 'r') as src:
    INGESTION_CLIENT.ingest_from_stream(src, ingestion_properties=INGESTION_PROPERTIES)

In [34]:
from datetime import datetime, timezone
p_time = "2024-03-14 00:00:00.0000000"
datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S.%f")

'2024-04-16 04:12:35.288579'

In [37]:
tenantid = "6babcaad-604b-40ac-a9d7-9fd97c0b779f"
clientid = "6e8d915a-460b-499c-a77f-10add633ef38"

In [ ]:
import adal
authority_url = 'https://login.microsoftonline.com/'+tenantid
context = adal.AuthenticationContext(authority_url)
token = context.acquire_token()

print(token["accessToken"])